# Load dependencies


In [1]:
import json
import os
import re

import numpy as np
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from huggingface_hub import InferenceClient
from scipy import stats
from statsmodels.stats import inter_rater as irr
from tqdm.auto import tqdm

tqdm.pandas()  # load tqdm's pandas support
pd.set_option("display.max_colwidth", None)

load_dotenv(find_dotenv())

True

# Helper functions


In [73]:
# Spearman's r
def calculate_spearman(df, col_1, col_2):
    return stats.spearmanr(df[col_1], df[col_2])


# Cohen's Kappa
def calculate_cohens_kappa(df, col_1, col_2):
    contingency_table = pd.crosstab(df[col_1], df[col_2], dropna=False)
    return irr.cohens_kappa(contingency_table)


# Cramér's V
def calculate_cramers_v(df, col_1, col_2):
    contingency_table = pd.crosstab(df[col_1], df[col_2], dropna=True)
    chi2 = stats.chi2_contingency(contingency_table)[0]
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape) - 1
    return np.sqrt(chi2 / (n * min_dim))


# calculate all metrics
def calculate_all_metrics(df, col_1, col_2):
    spearman_corr, _ = calculate_spearman(df, col_1, col_2)
    cohens_kappa = calculate_cohens_kappa(df, col_1, col_2)
    cramers_v = calculate_cramers_v(df, col_1, col_2)

    print(f"Spearman's r: {spearman_corr:.4f}")
    print(f"Cohen's Kappa: {cohens_kappa.kappa:.4f}")
    print(f"Cramér's V: {cramers_v:.4f}")

# Setup LLM client


In [19]:
repo_id = "meta-llama/Meta-Llama-3-8B-Instruct"

llm_client = InferenceClient(
    model=repo_id, timeout=120, token=os.environ["HUGGINGFACE_API_KEY"]
)

# test the client
llm_client.text_generation(prompt="How are you today?", max_new_tokens=20)

" I hope you're having a great day! I just wanted to check in and see how things are"

# LLM as a judge


## download dataset


In [20]:
if not os.path.exists("../data/llm_eval/feedback_train.json"):
    os.system(
        "!wget https://github.com/McGill-NLP/feedbackqa/raw/main/data/feedback_train.json -O ../data/llm_eval/feedback_train.json"
    )

In [5]:
# create a human evaluation dataset (~30 samples)
# to evaluate how reliable this LLM is as a judge
ratings = json.load(open("../data/llm_eval/feedback_train.json"))
ratings = pd.DataFrame(ratings)

In [6]:
# Retrieve the answers
ratings["answer"] = ratings["passage"].apply(
    lambda x: x["reference"]["section_content"]
)

# Create new columns for review_1 and review_2
ratings["review_1"] = ratings["rating"].apply(lambda x: x[0])
ratings["review_2"] = ratings["rating"].apply(lambda x: x[1] if len(x) > 1 else None)

# Create new columns for explanation_1 and explanation_2
ratings["explanation_1"] = ratings["feedback"].apply(lambda x: x[0])
ratings["explanation_2"] = ratings["feedback"].apply(
    lambda x: x[1] if len(x) > 1 else None
)

# Drop the original feedback and rating columns
ratings = ratings.drop(columns=["feedback", "rating"])

In [7]:
# map the review to an integer
conversion_dict = {"Excellent": 4, "Acceptable": 3, "Could be Improved": 2, "Bad": 1}
ratings["score_1"] = ratings["review_1"].map(conversion_dict)
ratings["score_2"] = ratings["review_2"].map(conversion_dict)

## measuring the correlation between two raters

- lets compare different metrics:
  - spearman's r: for measuring the correlation strength
  - kendall's tau: for measuring smaller samples, more robust to outliers
  - cohen's kappa: for measuring agreement
  - cramer's v: for general association (especially if the variables are treated as categorical rather than ordinal)

we will use spearman's r, cohen's kappa and cramer's v, since we have > 5k samples.


| Aspect                       | Spearman's r                                                                                                | Kendall's tau                                                                                                 | Cohen's kappa                                                                       | Cramér's V                                                                                                                        |
| ---------------------------- | ----------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------- |
| **Prerequisites**            | Ordinal or continuous variables                                                                             | Ordinal or continuous variables                                                                               | Categorical variables (nominal or ordinal)                                          | Categorical variables (nominal or ordinal)                                                                                        |
| **Assumptions**              | Monotonic relationship between variables                                                                    | Monotonic relationship between variables                                                                      | Independent observations                                                            | Independent observations                                                                                                          |
| **Sample Size Requirements** | Generally, n > 10                                                                                           | Generally, n > 10                                                                                             | Minimum 2 raters, at least 30 cases                                                 | No strict minimum, but larger samples provide more reliable results                                                               |
| **Testing Procedure**        | 1. Rank the data<br>2. Calculate Pearson correlation on ranks                                               | 1. Compare all pairs of observations<br>2. Calculate concordant and discordant pairs                          | 1. Create a contingency table<br>2. Calculate observed and expected agreement       | 1. Create a contingency table<br>2. Calculate chi-square statistic<br>3. Normalize based on sample size and degrees of freedom    |
| **Interpretation**           | -1 to 1, where:<br>-1: perfect negative correlation<br>0: no correlation<br>1: perfect positive correlation | -1 to 1, where:<br>-1: perfect negative correlation<br>0: no correlation<br>1: perfect positive correlation   | 0 to 1, where:<br>0: agreement by chance<br>1: perfect agreement                    | 0 to 1, where:<br>0: no association<br>1: perfect association                                                                     |
| **Use Cases**                | - Assessing strength and direction of monotonic relationships<br>- Ranking comparisons                      | - Assessing strength and direction of monotonic relationships<br>- Handling tied ranks better than Spearman's | - Measuring inter-rater agreement<br>- Assessing reliability of categorical ratings | - Measuring strength of association between categorical variables<br>- Useful when variables have different numbers of categories |
| **Strengths**                | - Robust to outliers<br>- Doesn't require normality assumption                                              | - More robust to outliers than Spearman's<br>- Better for small sample sizes and handling ties                | - Accounts for agreement by chance<br>- Widely used in reliability studies          | - Can be used with variables having different numbers of categories<br>- Provides a standardized measure of association           |
| **Limitations**              | - Doesn't capture non-monotonic relationships<br>- Can overestimate strength for small samples              | - Interpretation can be less intuitive than Spearman's<br>- Computationally intensive for large datasets      | - Sensitive to marginal distributions<br>- Doesn't consider degree of disagreement  | - Doesn't indicate direction of association<br>- Can be influenced by sample size                                                 |


In [8]:
len(ratings.score_1), len(ratings.score_2)  # number of samples

(5660, 5660)

In [57]:
calculate_all_metrics(ratings, col_1="score_1", col_2="score_2")

Spearman's r: 0.5609
Cohen's Kappa: 0.2838
Cramér's V: 0.3350


- Spearman's r (0.5609) suggests a moderate to strong correlation in the ranking order, meaning that as one rater's scores increase, the other rater's scores tend to increase as well.
- Cohen's Kappa (0.2838) suggests a only fair agreement between the raters, indicating substantial discrepancies between raters.
- Cramér's V: (0.3350) suggests that there is a moderate (not very strong) association between the two raters' categorization (without considering the ordinal nature)


In [10]:
# we examine instances where both raters reach consensus
ratings_where_raters_agree = ratings.loc[ratings["score_1"] == ratings["score_2"]]
examples = ratings_where_raters_agree.groupby("score_1").sample(7, random_state=1214)
examples["human_score"] = examples["score_1"]

print(f"the number of instances is: {len(examples)}")

# visualize 1 sample for each score
display(examples.groupby("human_score").first())

the number of instances is: 28


question  \
human_score                                                                          
1                                  What can I do to help people that are grieving?   
2              What protocols do workplaces need to follow to keep everyone safer?   
3                                      How soon can I apply for financial support?   
4            Should vulnerable children be expected to be in educational settings?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

## create the LLM judge

- task description
- scale description: minimum, maximum, value types (float here)
- explanation of the output format
- a beginning of an answer, to take the LLM by the hand as far as we can


In [21]:
JUDGE_PROMPT = """
You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer as a float on a scale of 0 to 10, where 0 means that the system_answer is not helpful at all, and 10 means that the answer completely and helpfully addresses the question.

Provide your feedback as follows:

Feedback:::
Total rating: (your rating, as a float between 0 and 10)

Now here are the question and answer.

Question: {question}
Answer: {answer}

Feedback:::
Total rating: """

In [22]:
examples["llm_judge"] = examples.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(question=x["question"], answer=x["answer"]),
        max_new_tokens=1000,
    ),
    axis=1,
)

  0%|          | 0/28 [00:00<?, ?it/s]

In [54]:
def extract_judge_score(answer: str, split_str: str = "\n\n") -> int:
    try:
        rating = answer.split(split_str)[0] if split_str in answer else answer
        # digit_groups = [el.strip() for el in re.findall(r"\d+(?:\.\d+)?", rating)]
        return float(rating)
    except Exception as e:
        print(e)
        return None


examples["llm_judge_score"] = examples["llm_judge"].apply(extract_judge_score)

# rescale the score given by the LLM on the same scale as the human score
examples["llm_judge_score"] = pd.cut(
    examples["llm_judge_score"],
    [0, 2.5, 5, 7.5, 10],
    labels=[1, 2, 3, 4],
    include_lowest=True,
)

In [74]:
calculate_all_metrics(examples, col_1="human_score", col_2="llm_judge_score")

Spearman's r: 0.6234
Cohen's Kappa: 0.1429
Cramér's V: 0.4792


## improve the LLM judge

- Leave more time for thought by adding an Evaluation field before the final answer.
- Use a small integer scale like 1-4 or 1-5 instead of a large float scale as we had previously.
- Provide an indicative scale for guidance.
- We even add a carrot to motivate the LLM!


In [80]:
IMPROVED_JUDGE_PROMPT = """
You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer on a scale of 1 to 4, where 1 means that the system_answer is not helpful at all, and 4 means that the system_answer completely and helpfully addresses the user_question.

Here is the scale you should use to build your answer:
1: The system_answer is terrible: completely irrelevant to the question asked, or very partial
2: The system_answer is mostly not helpful: misses some key aspects of the question
3: The system_answer is mostly helpful: provides support, but still could be improved
4: The system_answer is excellent: relevant, direct, detailed, and addresses all the concerns raised in the question

Provide your feedback as follows:

Feedback:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 4)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and answer.

Question: {question}
Answer: {answer}

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.
Feedback:::
Total rating: """

In [81]:
examples["llm_judge_improved"] = examples.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=IMPROVED_JUDGE_PROMPT.format(question=x["question"], answer=x["answer"]),
        max_new_tokens=500,
    ),
    axis=1,
)

  0%|          | 0/28 [00:00<?, ?it/s]

In [83]:
# extract the improved scores
examples["llm_judge_improved_score"] = examples["llm_judge_improved"].apply(
    lambda x: extract_judge_score(x, split_str="\n")
)

In [87]:
# scale the scores
examples["llm_judge_improved_score"] = pd.cut(
    examples["llm_judge_improved_score"],
    [0, 2.5, 5, 7.5, 10],
    labels=[1, 2, 3, 4],
    include_lowest=True,
)

# convert the data type to int for comparison
examples["llm_judge_improved_score"] = examples["llm_judge_improved_score"].astype(
    "int64"
)

In [104]:
errors = pd.concat(
    [
        examples.loc[
            examples["llm_judge_improved_score"] > examples["human_score"]
        ].head(1),
        examples.loc[
            examples["llm_judge_improved_score"] < examples["human_score"]
        ].head(2),
    ]
)

In [106]:
display(
    errors[
        [
            "question",
            "answer",
            "human_score",
            "explanation_1",
            "llm_judge_improved_score",
            "llm_judge_improved",
        ]
    ]
)

,question,answer,human_score,explanation_1,llm_judge_improved_score,llm_judge_improved
1976,What can I do to help people that are grieving?,"Taking care of yourself, your friends, and your family can help you cope with\nstress. Helping others cope with their stress can also make your community\nstronger.\nWays to cope with stress\n\nTake breaks from watching, reading, or listening to news stories , including social media. Hearing about the pandemic repeatedly can be upsetting.\nTake care of your body. \nTake deep breaths, stretch, or meditate.\nTry to eat healthy, well-balanced meals.\nExercise regularly, get plenty of sleep.\nAvoid alcohol and drugs.\n\n\nMake time to unwind. Try to do some other activities you enjoy.\nConnect with others. Talk with people you trust about your concerns and how you are feeling.\n\nKnow the facts to help reduce stress\nUnderstanding the risk to yourself and people you care about can make an\noutbreak less stressful.\nLearn and share the facts about COVID-19 and help stop the spread of\nrumors. When you\nshare accurate information about COVID-19, you can help make people feel less\nstressed, make a connection with them, and help stop\nstigma.\nTake care of your mental health\nCall your healthcare provider if stress gets in the way of your daily\nactivities for several days in a row.\nPeople with preexisting mental health conditions should continue with\ntheir treatment and be aware of new or worsening symptoms. Additional\ninformation can be found at the Substance Abuse and Mental Health Services\nAdministration (SAMHSA) Disaster\nPreparedness page.\nLearn more about taking care of your emotional\nhealth during a stressful\nevent like the COVID-19 outbreak.",1,The question is about others which the reply did not answer.,2,"3\nEvaluation: The system_answer provides some helpful information on how to cope with stress and grief, such as taking care of oneself, connecting with others, and understanding the facts about COVID-19. However, it does not directly address the question of what to do to help people who are grieving. The answer is mostly helpful, but could be improved by providing more specific and relevant information on how to support those who are grieving."
472,Can the covid19 event visa be granted to anyone?,The COVID-19 Pandemic event visa can only be granted to people in Australia.,2,"This information stated that the Covid-19 Pandemic event visa can be granted to people in Australia, however, it is not clear as to what groups of people in Australia are eligible for this visa.",1,"2\nEvaluation: The system_answer is mostly not helpful because it only answers a part of the question. The question asks if the visa can be granted to anyone, but the system_answer only mentions that it can only be granted to people in Australia, which is a specific group of people. The answer does not address the broader question of whether the visa can be granted to anyone in general."
670,What programs can assist busy childcare facilities?,"For a provider this payment and the base payment under the Relief Package are\nnot considered as revenue for GST purposes. This means providers will be able\nto show they satisfy the decline in income test for the purposes of the\nJobKeeper Payment provided they do not have income from other sources, such as\nbeing part of a larger entity like a non-government school or a not-for-profit\norganisation.\nWhere some of this revenue is then passed on to Family Day Care and In Home\nCare educators (based on contractual arrangements between the service and the\neducator) these monies are considered as revenue for GST purposes. As the\neducator is unlikely to receive more than 50 per cent of their fee revenue\nfrom the provider, they should be able to satisfy the decline in income test\nfor the JobKeeper Payment.",2,Gives some information on assistance programs but no references or contacts,1,"1\nEvaluation: The system_answer is terrible: completely irrelevant to the question a

# References

- [Using LLM-as-a-judge 🧑‍⚖️ for an automated and versatile evaluation](https://huggingface.co/learn/cookbook/llm_judge#2-create-our-llm-judge)
